# 2025年度 IoPサマースクール #8 演習2

気象衛星「ひまわり」の観測データ（2025年8月分）を使用し、  
**日射量を予測する機械学習モデル**を構築します。


In [164]:
# 使用するライブラリを読み込み
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    Concatenate, GlobalAveragePooling2D
)
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression



In [ ]:
# ひまわりデータ2025年8月分のダウンロード
!git clone https://github.com/ikunao/250909_SS8
!unzip ./250909_SS8/himawari.zip -d /content/


In [165]:
# ひまわりデータの読み込み
path_b03 = '/content/b03_japan/*.jpg'
path_b06 = '/content/b06_japan/*.jpg'
path_b13 = '/content/b13_japan/*.jpg'

x1 = []
x2 = []
x3 = []

for p in sorted(glob.glob(path_b03)):
    image = Image.open(p).convert('L')
    x1.append(np.array(image))

x1 = (x1 - np.mean(x1)) / (np.max(x1) - np.min(x1))

x = x1
x = x[0:360]     #8/1～8/15分のみ使用。

In [ ]:
# ひまわりデータの可視化

print(np.array(x).shape)
plt.imshow(x[5,50:150,:])  #5枚目の画像、Y方向50:150, X方向全部、最初のバンドを表示。

In [167]:
# 高知気象台環境情報データをデータフレーム格納

df = pd.read_csv(
    '/content/kisho_kochi.csv',
    encoding='shift_jis',
    parse_dates=['年月日時'],
    index_col='年月日時'
)
y = np.array(df["日射量"], dtype=np.float32)               #目的変数
w1 = np.array(df["晴天時日射量"], dtype=np.float32)        #説明変数
w1 = (w1 - np.mean(w1)) / (np.max(w1) - np.min(w1) + 1e-6)
w = w1

y = y[0:360]
w = w[0:360]

In [ ]:
# 環境情報の確認
print(df[:20])


In [ ]:
# 晴天時日射量と実際の日射量の相関

plt.figure(figsize=(6, 6))
plt.scatter(df["日射量"], df["晴天時日射量"], color='blue', label='Data')

min_val = min(df["日射量"].min(), df["晴天時日射量"].min())
max_val = max(df["日射量"].max(), df["晴天時日射量"].max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')

plt.xlabel('Calcurated PPFD')
plt.ylabel('Real PPFD')
plt.legend()
plt.show()


In [171]:
# 学習データと検証データに分離（前66% → train、後33% → val）
split_idx = int(len(x) * 0.66)
x_train, x_val = x[:split_idx], x[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]
w_train, w_val = w[:split_idx], w[split_idx:]

In [172]:
# 画像特徴と数値情報を融合した軽量CNN回帰ネットワーク

# 画像の畳み込み
image_input = Input(shape=(200,400, 1))
v = Conv2D(8, (3, 3), activation='relu', padding='same', dilation_rate=3)(image_input)
v = MaxPooling2D((2, 2))(v)
v = Conv2D(8, (3, 3), activation='relu', padding='same', dilation_rate=3)(v)
v = GlobalAveragePooling2D()(v)
v = Dense(8, activation='relu')(v)

# 数値データのNN
meta_input = Input(shape=(1,))    #インプットデータのshapeに合わせる
m = Dense(8, activation='relu')(meta_input)
m = Dense(8, activation='relu')(m)

# 画像と数値を結合
combined = Concatenate()([v, m])
output = Dense(1, activation='linear')(combined)

# モデル構築
model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')


In [ ]:
# モデル学習
history = model.fit(
    [x_train, w_train], y_train,
    validation_data=([x_val, w_val], y_val),
    epochs=10,
    batch_size=8
)

In [ ]:
# 学習曲線
plt.figure(figsize=(6, 6))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# 予測値
y_pred = model.predict([x_val,w_val]).flatten()

# 相関係数
r_value = np.corrcoef(y_val, y_pred)[0, 1]

# 回帰直線
reg_model = LinearRegression()
reg_model.fit(y_val.reshape(-1, 1), y_pred)
reg_line = reg_model.predict(y_val.reshape(-1, 1))

# 描画
plt.figure(figsize=(6, 6))
plt.scatter(x=y_val, y=y_pred, color='blue', label='Predicted vs True')
plt.plot(y_val, reg_line, color='red', label=f'Regression Line (R={r_value:.2f})')
plt.xlabel('True PPFD')
plt.ylabel('Predicted PPFD')
plt.legend()
plt.show()

In [ ]:
# 時系列結果の表示（学習に使用しなかった後半）
y_pred = model.predict([x_val,w_val]).flatten()
time_index = np.arange(len(y_val))

# グラフの描画
plt.figure(figsize=(10, 4))
plt.plot(time_index, y_val, label='True PPFD', color='blue', linewidth=2)
plt.plot(time_index, y_pred, label='Predicted PPFD', color='orange', linestyle='--', linewidth=2)

plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# 時系列結果の表示（全期間）
y_pred = model.predict([x,w]).flatten()
time_index = np.arange(len(y))

plt.figure(figsize=(10, 4))
plt.plot(time_index, y, label='True PPFD', color='blue', linewidth=2)
plt.plot(time_index, y_pred, label='Predicted PPFD', color='orange', linestyle='--', linewidth=2)
